In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn import neighbors
from math import sqrt

from sklearn import metrics
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
data = pd.read_csv(r"C:\Users\Anuj\SideChannelAttack\Extracted_features_from_magnetometer_readings_Cleaned.csv")

In [3]:
data

,Unnamed: 0,Magnitude__abs_energy,Magnitude__absolute_sum_of_changes,"Magnitude__agg_autocorrelation__f_agg_""mean""__maxlag_40","Magnitude__agg_autocorrelation__f_agg_""median""__maxlag_40","Magnitude__agg_autocorrelation__f_agg_""var""__maxlag_40","Magnitude__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""max""","Magnitude__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""","Magnitude__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""","Magnitude__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""",...,Z__symmetry_looking__r_0.9500000000000001,Z__time_reversal_asymmetry_statistic__lag_1,Z__time_reversal_asymmetry_statistic__lag_2,Z__time_reversal_asymmetry_statistic__lag_3,Z__value_count__value_-1,Z__value_count__value_0,Z__value_count__value_1,Z__variance,Z__variance_larger_than_standard_deviation,Z__variation_coefficient
0,1,38196.839416,15.730598,-0.041109,-0.049222,0.062105,25.273309,24.878358,24.414126,0.055069,...,1.0,3.908693,2.319009,-0.012910,0.0,0.0,0.0,0.255944,0.0,-0.030803
1,1,38350.589049,15.159211,-0.034701,0.023161,0.108069,26.208657,25.681117,25.049185,0.123030,...,1.0,9.952164,44.273556,67.561471,0.0,0.0,0.0,0.304611,0.0,-0.033825
2,1,37748.777465,16.175135,-0.018500,-0.061378,0.075548,25.615439,25.073784,24.754291,0.077592,...,1.0,17.494609,35.507800,47.832088,0.0,0.0,0.0,0.191939,0.0,-0.028172
3,1,37025.388548,14.407682,-0.105366,-0.099327,0.131342,25.637383,25.158538,24.466985,0.120080,...,1.0,1.831162,-8.068637,-11.245012,0.0,0.0,0.0,0.174224,0.0,-0.027819
4,1,35471.342041,15.680723,-0.106679,-0.148544,0.336794,25.476993,25.213351,24.818035,-0.033318,...,1.0,8.436691,37.120551,72.154189,0.0,0.0,0.0,0.620451,0.0,-0.055758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,3,33663.784727,13.843845,-0.054119,-0.068446,0.100715,24.389290,23.992663,23.430376,0.067722,...,1.0,14.071379,27.909304,44.509738,0.0,0.0,0.0,0.237459,0.0,-0.030746
73,3,32686.258545,17.908074,-0.019959,0.010272,0.042416,23.943848,23.326531,22.597841,0.203778,...,1.0,-18.243557,-28.525421,-24.200967,0.0,0.0,0.0,0.162419,0.0,-0.025441
74,3,30713.180206,14.817158,-0.022046,0.008090,0.092820,22.845247,22.131015,21.487709,0.204369,...,1.0,-11.814355,-24.754849,-38.964664,0.0,0.0,0.0,0.424416,0.0,-0.041633
75,3,33171.398009,16.372867,-0.020215,-0.023867,0.033167,23.732554,23.334745,22.804081,0.072618,...,1.0,25.580603,48.576762,74.435868,0.0,0.0,0.0,0.346331,0.0,-0.037563


In [4]:
X = data.drop("Unnamed: 0", 1)
y = data["Unnamed: 0"]

In [5]:
X

,Magnitude__abs_energy,Magnitude__absolute_sum_of_changes,"Magnitude__agg_autocorrelation__f_agg_""mean""__maxlag_40","Magnitude__agg_autocorrelation__f_agg_""median""__maxlag_40","Magnitude__agg_autocorrelation__f_agg_""var""__maxlag_40","Magnitude__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""max""","Magnitude__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""","Magnitude__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""","Magnitude__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""","Magnitude__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""max""",...,Z__symmetry_looking__r_0.9500000000000001,Z__time_reversal_asymmetry_statistic__lag_1,Z__time_reversal_asymmetry_statistic__lag_2,Z__time_reversal_asymmetry_statistic__lag_3,Z__value_count__value_-1,Z__value_count__value_0,Z__value_count__value_1,Z__variance,Z__variance_larger_than_standard_deviation,Z__variation_coefficient
0,38196.839416,15.730598,-0.041109,-0.049222,0.062105,25.273309,24.878358,24.414126,0.055069,26.177241,...,1.0,3.908693,2.319009,-0.012910,0.0,0.0,0.0,0.255944,0.0,-0.030803
1,38350.589049,15.159211,-0.034701,0.023161,0.108069,26.208657,25.681117,25.049185,0.123030,26.321862,...,1.0,9.952164,44.273556,67.561471,0.0,0.0,0.0,0.304611,0.0,-0.033825
2,37748.777465,16.175135,-0.018500,-0.061378,0.075548,25.615439,25.073784,24.754291,0.077592,26.240861,...,1.0,17.494609,35.507800,47.832088,0.0,0.0,0.0,0.191939,0.0,-0.028172
3,37025.388548,14.407682,-0.105366,-0.099327,0.131342,25.637383,25.158538,24.466985,0.120080,26.018839,...,1.0,1.831162,-8.068637,-11.245012,0.0,0.0,0.0,0.174224,0.0,-0.027819
4,35471.342041,15.680723,-0.106679,-0.148544,0.336794,25.476993,25.213351,24.818035,-0.033318,25.502918,...,1.0,8.436691,37.120551,72.154189,0.0,0.0,0.0,0.620451,0.0,-0.055758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,33663.784727,13.843845,-0.054119,-0.068446,0.100715,24.389290,23.992663,23.430376,0.067722,24.544083,...,1.0,14.071379,27.909304,44.509738,0.0,0.0,0.0,0.237459,0.0,-0.030746
73,32686.258545,17.908074,-0.019959,0.010272,0.042416,23.943848,23.326531,22.597841,0.203778,24.271340,...,1.0,-18.243557,-28.525421,-24.200967,0.0,0.0,0.0,0.162419,0.0,-0.025441
74,30713.180206,14.817158,-0.022046,0.008090,0.092820,22.845247,22.131015,21.487709,0.204369,24.033752,...,1.0,-11.814355,-24.754849,-38.964664,0.0,0.0,0.0,0.424416,0.0,-0.041633
75,33171.398009,16.372867,-0.020215,-0.023867,0.033167,23.732554,23.334745,22.804081,0.072618,24.000750,...,1.0,25.580603,48.576762,74.435868,0.0,0.0,0.0,0.346331,0.0,-0.037563


In [6]:
y

0     1
1     1
2     1
3     1
4     1
     ..
72    3
73    3
74    3
75    3
76    3
Name: Unnamed: 0, Length: 77, dtype: int64

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 4)

In [8]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(53, 1943) (24, 1943) (53,) (24,)


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

## pytcart Random Forest Model

In [10]:
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute


relevant_features = set()

for label in y.unique():
    y_train_binary = y_train == label
    X_train_filtered = select_features(X_train, y_train_binary)
    print("Number of relevant features for class {}: {}/{}".format(label, X_train_filtered.shape[1], X_train.shape[1]))
    relevant_features = relevant_features.union(set(X_train_filtered.columns))

Number of relevant features for class 1: 136/1943
Number of relevant features for class 2: 172/1943
Number of relevant features for class 3: 198/1943


In [11]:
len(relevant_features)

303

In [12]:
relevant_features

{'Magnitude__abs_energy',
 'Magnitude__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"max"',
 'Magnitude__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"mean"',
 'Magnitude__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"min"',
 'Magnitude__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"max"',
 'Magnitude__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"mean"',
 'Magnitude__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"min"',
 'Magnitude__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"max"',
 'Magnitude__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"mean"',
 'Magnitude__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"min"',
 'Magnitude__c3__lag_1',
 'Magnitude__c3__lag_2',
 'Magnitude__c3__lag_3',
 'Magnitude__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)',
 'Magnitude__cwt_coefficients__coeff_10__w_10__widths_(2, 5, 10, 20)',
 'Magnitude__cwt_coefficients__coeff_10__w_20__widths_(2, 5, 10, 

In [13]:
X_train_filtered = X_train[list(relevant_features)]
X_test_filtered = X_test[list(relevant_features)]

In [14]:
X_train_filtered.head()

,"Magnitude__cwt_coefficients__coeff_12__w_20__widths_(2, 5, 10, 20)",Magnitude__quantile__q_0.2,"Y__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""mean""",Y__abs_energy,"Z__cwt_coefficients__coeff_14__w_5__widths_(2, 5, 10, 20)","Magnitude__cwt_coefficients__coeff_13__w_10__widths_(2, 5, 10, 20)",Magnitude__max_langevin_fixed_point__m_3__r_30,Magnitude__variation_coefficient,"Magnitude__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)",Magnitude__quantile__q_0.7,...,Y__quantile__q_0.6,Y__c3__lag_1,"Y__cwt_coefficients__coeff_8__w_20__widths_(2, 5, 10, 20)","Magnitude__cwt_coefficients__coeff_3__w_5__widths_(2, 5, 10, 20)","Z__cwt_coefficients__coeff_12__w_20__widths_(2, 5, 10, 20)","Magnitude__cwt_coefficients__coeff_13__w_5__widths_(2, 5, 10, 20)","Magnitude__cwt_coefficients__coeff_2__w_5__widths_(2, 5, 10, 20)","Z__cwt_coefficients__coeff_8__w_10__widths_(2, 5, 10, 20)","Z__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","Magnitude__cwt_coefficients__coeff_11__w_5__widths_(2, 5, 10, 20)"
11,88.552251,22.625790,-35.470800,75319.270086,-0.155645,37.371661,24.718231,0.019268,7.566716,23.186683,...,-35.375999,-44462.318275,-118.547399,22.396659,-54.617483,3.029428,16.560206,-23.167844,-5.185971,8.163940
39,111.001716,28.469381,-31.972799,61085.892742,-2.264678,48.008305,35.185220,0.016744,9.354528,28.992371,...,-31.740000,-32503.986207,-105.892803,27.433824,-80.647667,5.303511,20.068729,-34.632115,-6.801098,10.976014
22,96.099408,16.848906,-32.742493,13144.057067,-1.998304,40.983100,19.859367,0.036891,6.851835,17.993616,...,-32.964000,-36378.931451,-62.133700,37.912559,-67.016049,4.490891,33.464111,-23.620446,-3.189357,25.743303
18,78.301010,19.946839,-33.362400,67154.279591,0.138938,32.550310,20.869758,0.027717,7.841037,20.830988,...,-33.420000,-37457.974612,-110.483957,20.781643,-50.109879,2.347463,15.311215,-21.358394,-5.236711,6.551383
64,84.630975,21.765487,-31.649999,59985.851360,-2.374603,36.339911,22.252915,0.021818,6.697887,22.367733,...,-31.439999,-31600.402723,-105.576487,20.433804,-59.522752,4.253375,14.932625,-26.198842,-4.536204,8.519190


In [15]:
X_test_filtered.head()

,"Magnitude__cwt_coefficients__coeff_12__w_20__widths_(2, 5, 10, 20)",Magnitude__quantile__q_0.2,"Y__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""mean""",Y__abs_energy,"Z__cwt_coefficients__coeff_14__w_5__widths_(2, 5, 10, 20)","Magnitude__cwt_coefficients__coeff_13__w_10__widths_(2, 5, 10, 20)",Magnitude__max_langevin_fixed_point__m_3__r_30,Magnitude__variation_coefficient,"Magnitude__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)",Magnitude__quantile__q_0.7,...,Y__quantile__q_0.6,Y__c3__lag_1,"Y__cwt_coefficients__coeff_8__w_20__widths_(2, 5, 10, 20)","Magnitude__cwt_coefficients__coeff_3__w_5__widths_(2, 5, 10, 20)","Z__cwt_coefficients__coeff_12__w_20__widths_(2, 5, 10, 20)","Magnitude__cwt_coefficients__coeff_13__w_5__widths_(2, 5, 10, 20)","Magnitude__cwt_coefficients__coeff_2__w_5__widths_(2, 5, 10, 20)","Z__cwt_coefficients__coeff_8__w_10__widths_(2, 5, 10, 20)","Z__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","Magnitude__cwt_coefficients__coeff_11__w_5__widths_(2, 5, 10, 20)"
49,96.099408,27.084534,-32.742493,11773.144664,-1.998304,40.983100,27.974798,0.007995,12.509127,27.422843,...,-33.935999,-40285.467924,-45.117392,58.912046,-67.016049,4.490891,51.900706,-37.855972,-10.291623,9.816197
66,82.805359,21.453521,-31.225200,58911.640558,-1.555433,34.578761,22.333776,0.032055,6.511355,22.572732,...,-31.296000,-30763.161840,-102.580424,20.221128,-55.931363,4.213361,14.774353,-24.359796,-4.140611,8.179132
60,81.627895,20.827145,-31.629600,60726.414815,-2.035691,34.804923,21.692647,0.021631,6.275973,21.305560,...,-31.560000,-32186.045604,-103.521570,20.865667,-57.486767,3.691265,15.165104,-27.173498,-4.705110,9.292920
48,99.312644,25.932417,-34.129199,69831.735741,-2.576215,40.674800,27.238631,0.030573,7.664144,26.962699,...,-33.996000,-39691.785372,-112.110346,25.457732,-80.427124,5.015799,19.039872,-36.470620,-6.140793,10.933197
14,90.453914,23.301244,-35.197200,74824.907646,-1.550137,37.425958,24.677986,0.023780,8.064193,24.110235,...,-35.219997,-44024.241628,-116.376068,22.701348,-53.016832,3.500157,16.687377,-23.454513,-4.736302,7.957544


In [67]:
classifier = RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=60, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=5635,
                       verbose=0, warm_start=False)

In [74]:
classifier.fit(X_train_filtered, y_train)

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=60,
                       min_samples_leaf=4, min_samples_split=5, n_estimators=40,
                       random_state=5635)

In [75]:
y_pred = classifier.predict(X_test_filtered)

In [76]:
print(accuracy_score(y_test,y_pred))

1.0


In [77]:
print(confusion_matrix(y_test,y_pred))

[[9 0 0]
 [0 9 0]
 [0 0 6]]


In [78]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00         9
           3       1.00      1.00      1.00         6

    accuracy                           1.00        24
   macro avg       1.00      1.00      1.00        24
weighted avg       1.00      1.00      1.00        24



## Decision Tree

In [83]:
from sklearn import tree
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [84]:
classifier  = tree.DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=150, max_features=251, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=6624, splitter='best')

In [85]:
classifier.fit(X_train_filtered, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=150, max_features=251,
                       min_samples_leaf=4, random_state=6624)

In [86]:
y_pred = classifier.predict(X_test_filtered)

In [87]:
print(accuracy_score(y_test,y_pred))

0.9583333333333334


In [88]:
print(confusion_matrix(y_test,y_pred))

[[9 0 0]
 [1 8 0]
 [0 0 6]]


In [89]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.90      1.00      0.95         9
           2       1.00      0.89      0.94         9
           3       1.00      1.00      1.00         6

    accuracy                           0.96        24
   macro avg       0.97      0.96      0.96        24
weighted avg       0.96      0.96      0.96        24



## Sklearn Random Forest

In [92]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train_filtered,y_train)

y_pred=clf.predict(X_test_filtered)

error = sqrt(metrics.mean_squared_error( y_test, y_pred)) 
print('RMSE value of the KNN Model is:', error)

print(accuracy_score(y_test,y_pred))

print(confusion_matrix(y_test,y_pred))

print(classification_report(y_test,y_pred))

RMSE value of the KNN Model is: 0.0
1.0
[[9 0 0]
 [0 9 0]
 [0 0 6]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00         9
           3       1.00      1.00      1.00         6

    accuracy                           1.00        24
   macro avg       1.00      1.00      1.00        24
weighted avg       1.00      1.00      1.00        24



## KNN

In [82]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 9)

knn.fit(X_train_filtered, y_train)

y_pred = knn.predict(X_test_filtered)

print("Test Score: ", knn.score(X_test_filtered, y_test))

Test Score:  0.875


## SVC

In [109]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

cv_grid_svc = GridSearchCV(SVC(), param_grid = {
    'degree':[3,4],
    'shrinking':[True,False],
    'probability':[True,False]
}, cv=10)

cv_grid_svc.fit(X_train_filtered, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'degree': [3, 4], 'probability': [True, False],
                         'shrinking': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [112]:
bestclassifier_svc=cv_grid_svc.best_estimator_

y_predict_svc = bestclassifier_svc.predict(X_test_filtered)
accuracy = metrics.accuracy_score(y_test,y_predict_svc)
print(cv_grid_svc.best_params_)
print('Accuracy of the best classifier after CV is %.3f%%' % (accuracy*100))

{'degree': 3, 'probability': True, 'shrinking': True}
Accuracy of the best classifier after CV is 87.500%


## Random Forest

In [114]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

cv_grid_rf = GridSearchCV(estimator = RandomForestClassifier(), param_grid = {
    'n_estimators': range(10, 100, 10),
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}, cv=10)

cv_grid_rf.fit(X_train_filtered, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rand

In [115]:
bestclassifier_rf=cv_grid_rf.best_estimator_

y_predict_rf = bestclassifier_rf.predict(X_test_filtered)
accuracy = metrics.accuracy_score(y_test,y_predict_rf)
print(cv_grid_rf.best_params_)
print('Accuracy of the best classifier after CV is %.3f%%' % (accuracy*100))

{'criterion': 'gini', 'max_depth': 4, 'max_features': 'auto', 'n_estimators': 30}
Accuracy of the best classifier after CV is 95.833%


## Logistic Regression

In [116]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

cv_grid_lr = GridSearchCV(estimator = LogisticRegression(), param_grid = {
    'C': np.logspace(0, 4, 10),
    'penalty': ['l1', 'l2'],
}, cv=10)

cv_grid_lr.fit(X_train_filtered, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
           

In [118]:
bestclassifier_lr=cv_grid_lr.best_estimator_

y_predict_lr = bestclassifier_lr.predict(X_test_filtered)
accuracy = metrics.accuracy_score(y_test,y_predict_lr)
print(cv_grid_lr.best_params_)
print("NUMBER OF ERRORS: " + str(sum(y_predict_lr != y_test)))
print("ERROR RATE: " + str(1 - sum(y_predict_lr == y_test) / float(len(y_predict_lr))) + "%")
print('Accuracy of the best classifier after CV is %.3f%%' % (accuracy*100))

{'C': 1.0, 'penalty': 'l2'}
NUMBER OF ERRORS: 1
ERROR RATE: 0.04166666666666663%
Accuracy of the best classifier after CV is 95.833%


## KNN

In [119]:
cv_grid_knn = GridSearchCV(estimator = KNeighborsClassifier(), param_grid = {
    'n_neighbors':[4,5,6,7],
    'leaf_size':[1,3,5],
    'algorithm':['auto', 'kd_tree']
}, cv=10)

cv_grid_knn.fit(X_train_filtered, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'algorithm': ['auto', 'kd_tree'],
                         'leaf_size': [1, 3, 5], 'n_neighbors': [4, 5, 6, 7]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [120]:
bestclassifier_knn=cv_grid_knn.best_estimator_

y_predict_knn = bestclassifier_knn.predict(X_test_filtered)
accuracy = metrics.accuracy_score(y_test,y_predict_knn)
print(cv_grid_knn.best_params_)
print("NUMBER OF ERRORS: " + str(sum(y_predict_knn != y_test)))
print("ERROR RATE: " + str(1 - sum(y_predict_knn == y_test) / float(len(y_predict_knn))) + "%")
print('Accuracy of the best classifier after CV is %.3f%%' % (accuracy*100))

{'algorithm': 'auto', 'leaf_size': 1, 'n_neighbors': 4}
NUMBER OF ERRORS: 1
ERROR RATE: 0.04166666666666663%
Accuracy of the best classifier after CV is 95.833%


## XGBOOST

In [122]:
from xgboost import XGBClassifier

cv_grid_xb = GridSearchCV(estimator = XGBClassifier(), param_grid = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(10, 100, 10),
    'learning_rate': [0.1, 0.01, 0.05]
}, cv=10)

cv_grid_xb.fit(X_train_filtered, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_esti...
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     validate_parameters=None, 

In [123]:
bestclassifier_xb=cv_grid_xb.best_estimator_

y_predict_xb = bestclassifier_xb.predict(X_test_filtered)
accuracy = metrics.accuracy_score(y_test,y_predict_xb)
print(cv_grid_xb.best_params_)
print("NUMBER OF ERRORS: " + str(sum(y_predict_xb != y_test)))
print("ERROR RATE: " + str(1 - sum(y_predict_xb == y_test) / float(len(y_predict_xb))) + "%")
print('Accuracy of the best classifier after CV is %.3f%%' % (accuracy*100))

{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 10}
NUMBER OF ERRORS: 0
ERROR RATE: 0.0%
Accuracy of the best classifier after CV is 100.000%
